In [1]:
import os
import sys
sys.path.append('../examples')
sys.path.append('../jobs')

from tqdm import trange

import torch
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config

from generate_with_calibration import get_lookahead_entropies

import logging
logging.getLogger('transformers.tokenization_utils').setLevel(logging.ERROR)

To use data.metrics please install scikit-learn. See https://scikit-learn.org/stable/index.html


In [ ]:
# old code 

#     total_loss.backward()
    
#     print('Minimizing CEL.')
#     oldloss = total_loss.clone().item()
#     for _ in range(iters):
#         total_loss = 0.0
#         optimizer.step()
#         with open(path) as fp:
#             for line in fp:
#                 context = tokenizer.encode(line)
#                 total_loss += CEL(model, context, alpha, vocab_size,
#                 top_k=top_k,
#                 batch_size=512, 
#                 device=device,
#                 require_grads=False).item()
#         print(f'Alpha: {alpha.item()}. Total Loss: {total_loss}')
#         if total_loss >= oldloss:
#             break
    
#     return alpha

#     return total_CEL
#     else:
#         with torch.no_grad():
#             total_CEL = torch.tensor([0.0], device=device)
#         for i in range(1, N):
#             context_i = torch.tensor(context[:i], dtype = torch.long, device=device).unsqueeze(0)
#             lookahead_ents = get_lookahead_entropies(
#                 model = model,
#                 context = context_i[0],
#                 batch_size = batch_size,
#                 vocab_size = vocab_size,
#                 device = device
#             ).detach()
            
#             inputs = {'input_ids': context_i}
#             outputs = model(**inputs)
#             next_logits = outputs[0][:, -1, :].detach()

#             calibrated_next_logits = next_logits - alpha * lookahead_ents
#             # print(calibrated_next_logits)
#             next_probs = F.softmax(calibrated_next_logits, dim=-1)
#             # print(next_probs)
#             next_word = context[i]
#             # sanity check this! make sure argmaxes are somewhat retained!!
#             print(torch.max(next_probs, 1))
#             print(next_word)
#             print(next_probs[0][next_word])
#             total_CEL += next_probs[0][context[i]]

#         return total_CEL

#     for i in range(iters):
#         total_loss = torch.tensor([0.0], device=device)
        
#         optimizer.zero_grad()
#         total_loss.backward()
#         optimizer.step()
#         print(f'Iter: {i}. Alpha: {alpha.item()}. Total Loss: {total_loss.item()}. Old Loss: {old_loss}')
        
#         if total_loss.item() >= old_loss:
#             break
#         old_loss = total_loss.item()
    
#     return alpha

# def calibrate(model, tokenizer, path, vocab_size, batch_size=512, top_k=0, threshold=1e-7, device='cuda'):
#     alpha = torch.randn(1, requires_grad=True, device=device)
#     total_loss = CEL(model, tokenizer, path, alpha, vocab_size, batch_size, top_k, device)
#     print(f'Total loss: {total_loss.item()}. Alpha: {alpha.item()}')
    
#     while total_loss.item() > threshold:
#         total_loss.backward()
#         alpha.data -= (total_loss / alpha.grad).data
#         alpha.grad.data.zero_()
#         total_loss = CEL(model, tokenizer, path, alpha, vocab_size, batch_size, top_k, device)
#         print(f'Total loss: {total_loss.item()}. Alpha: {alpha.item()}')
    
#     return alpha

# def CEL(model, tokenizer, path, alpha, vocab_size, batch_size=512, top_k = 0, device='cuda'):
#     def CELHelper(context, total_CEL):
#         N = len(context)
#         for i in trange(1, N):
#             context_i = torch.tensor(context[:i], dtype = torch.long, device=device).unsqueeze(0)
#             lookahead_ents = get_lookahead_entropies(
#                 model = model,
#                 context = context_i[0],
#                 batch_size = batch_size,
#                 vocab_size = vocab_size,
#                 device = device
#             ).detach()

#             inputs = {'input_ids': context_i}
#             outputs = model(**inputs)
#             next_logits = outputs[0][:, -1, :].detach()

#             calibrated_next_logits = next_logits - alpha * lookahead_ents
#             next_probs = F.softmax(calibrated_next_logits, dim=-1)
#             total_CEL += next_probs[0][context[i]]
            
#     total_CEL = torch.tensor([0.0], device=device)

#     with open(path) as fp:
#         for line in fp:
#             context = tokenizer.encode(line)
#             CELHelper(context, total_CEL)
    
#     return total_CEL

#     def CELHelper(context, total_CEL):
#         N = len(context)
#         for i in trange(1, N):
#             context_i = torch.tensor(context[:i], dtype = torch.long, device=device).unsqueeze(0)
#             lookahead_ents = get_lookahead_entropies(
#                 model = model,
#                 context = context_i[0],
#                 batch_size = batch_size,
#                 vocab_size = vocab_size,
#                 device = device
#             ).detach().cpu()

#             inputs = {'input_ids': context_i}
#             outputs = model(**inputs)
#             next_logits = outputs[0][:, -1, :].detach().cpu()
            
#             total_CEL += F.softmax(next_logits - alpha * lookahead_ents, dim=-1)[0][context[i]]
            
#     total_CEL = torch.tensor([0.0])

#     with open(path) as fp:
#         for line in fp:
#             context = tokenizer.encode(line)
#             CELHelper(context, total_CEL)
    
#     return total_CEL

In [2]:
def set_seed(seed=42, n_gpu=0):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)

# setup cell 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpus = torch.cuda.device_count()

set_seed()
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.to(device)
model.eval()

vocab_size = tokenizer.vocab_size

print(f'VOCAB SIZE: {vocab_size}')
print(f'DEVICE: {device}')
print(f'N GPUS: {n_gpus}')

12/08/2019 11:28:49 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-config.json from cache at /u/myhu/.cache/torch/transformers/4be02c5697d91738003fb1685c9872f284166aa32e061576bbe6aaeb95649fcf.085d5f6a8e7812ea05ff0e6ed0645ab2e75d80387ad55c1ad9806ee70d272f80
12/08/2019 11:28:49 - INFO - transformers.configuration_utils -   Model config {
  "attn_pdrop": 0.1,
  "embd_pdrop": 0.1,
  "finetuning_task": null,
  "initializer_range": 0.02,
  "is_decoder": false,
  "layer_norm_epsilon": 1e-05,
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_layer": 12,
  "n_positions": 1024,
  "num_labels": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torchscript": false,
  "use_bf

VOCAB SIZE: 50257
DEVICE: cuda
N GPUS: 1


In [5]:
def calibrate(model, tokenizer, path, vocab_size, batch_size=512, top_k=0, threshold=1e-7, device='cpu'):
    alpha = torch.randn(1, requires_grad=True)
    total_loss = CEL(model, tokenizer, path, alpha, vocab_size, batch_size, top_k, device)
    print(f'Total loss: {total_loss.item()}. Alpha: {alpha.item()}')
    
    while total_loss.item() > threshold:
        total_loss.backward()
        alpha.data -= (total_loss / alpha.grad).data
        alpha.grad.data.zero_()
        total_loss = CEL(model, tokenizer, path, alpha, vocab_size, batch_size, top_k, device)
        print(f'Total loss: {total_loss.item()}. Alpha: {alpha.item()}')
    
    return alpha

def CEL(model, tokenizer, path, alpha, vocab_size, batch_size=512, top_k=0, device='cpu'):    
    def CELHelper(context):
        N = len(context)
        Pr = torch.zeros(N-1)
        H = torch.zeros(N-1)
        S = torch.zeros(N-1)
        with torch.no_grad():
            for i in trange(1, N):
                context_i = torch.tensor(context[:i], dtype = torch.long, device=device).unsqueeze(0)
                inputs = {'input_ids': context_i}
                outputs = model(**inputs)
                next_logits = outputs[0][:, -1, :]

                if top_k == 0:
                    candidates = None 
                else:
                    candidates = torch.argsort(next_logits[0], descending=True,)[:top_k]

                lookahead_ents = get_lookahead_entropies(
                    model = model,
                    context = context_i[0],
                    batch_size = batch_size,
                    vocab_size = vocab_size,
                    candidates = candidates,
                    device = device
                )

                next_probs = F.softmax(next_logits, dim=-1)[0]
                
                if top_k != 0:
                # replace uncomputed entropies with average (for centered adjustment)
                    top_average_ent = lookahead_ents[lookahead_ents != -1].mean()
                    lookahead_ents[lookahead_ents != -1] = top_average_ent

                # cache useful values
                next_word = context[i]
                Pr[i-1] = next_probs[next_word]
                H[i-1] = lookahead_ents[next_word]
                S[i-1] = torch.dot(next_probs, torch.exp(lookahead_ents))
    
        Za = S * torch.exp(-alpha)
        return torch.sum(Pr * torch.exp(-alpha * H) / Za)
    
    total_CEL = torch.tensor([0.0])

    with open(path) as fp:
        for line in fp:
            context = tokenizer.encode(line)
            total_CEL += CELHelper(context)
    
    return total_CEL

In [ ]:
alpha = calibrate(
    model=model, 
    tokenizer=tokenizer, 
    path='../training_data/gbw/training/news1-head100',
    # path='../training_data/cats.txt',
    # path='../training_data/unicorn.txt',
    vocab_size=vocab_size, 
    batch_size = 128, 
    top_k=128, 
    threshold=1e-6,
    device='cuda'
)

  7%|▋         | 4/54 [00:00<00:01, 32.45it/s]

Total loss: 1313.0035400390625. Alpha: 0.2690553665161133


  7%|▋         | 4/54 [00:00<00:01, 31.99it/s]

Total loss: 485.4585876464844. Alpha: -0.23193234205245972


  7%|▋         | 4/54 [00:00<00:01, 31.34it/s]

...

  7%|▋         | 4/54 [00:00<00:01, 32.50it/s]

Total loss: 187.89088439941406. Alpha: -0.7434996366500854


  7%|▋         | 4/54 [00:00<00:01, 32.55it/s]

Total loss: 154.18508911132812. Alpha: -1.3617756366729736


 18%|█▊        | 9/51 [00:00<00:01, 24.54it/s]